In [ ]:
import cv2
import pandas as pd
import os
import time
import numpy as np
import easyocr
import string

In [ ]:
reader = easyocr.Reader(['en'])

In [ ]:
# Validating and Formatting detected text according to the USA Registration Number Format
dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}


def read_license_plate(license_plate_crop):

    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

        if license_complies_format(text):
            return format_license(text), score

    return None, None

def license_complies_format(text):

    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False
    

def format_license(text):

    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_

In [ ]:
# Main 
dataframe = {'Car ID': [], 'License Plate Number': [], 'Frame Number': [], 'Confidence Level': []}
df = pd.DataFrame(dataframe)

processed_frames = set()
frames_folder = "Detected Plate Frames"

while True:
    frames_to_process = [filename for filename in os.listdir(frames_folder) if filename.endswith(".jpg") and filename not in processed_frames]

    if not frames_to_process:
            print("All frames processed. Exiting the script.")
            break
    
    for filename in frames_to_process:
        frame_no = int(filename[1:filename.index('_')])
        car_id = int(filename[filename.index('D')+1:filename.index('.')])

        frame_path = os.path.join(frames_folder, filename)
        frame = cv2.imread(frame_path)

        sharpening_filter = np.array([[-1, -1, -1],
                              [-1,  10, -1],
                              [-1, -1, -1]])
        frame = cv2.filter2D(frame, -1, sharpening_filter)
        
        frame = cv2.fastNlMeansDenoisingColored(frame, None, h = 30, hColor = 16, templateWindowSize = 16, searchWindowSize = 10)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        scaling_factor = 2.3
        scaled_frame = cv2.resize(frame, None, fx = scaling_factor, fy = scaling_factor, interpolation = cv2.INTER_LINEAR)
        
        frame = cv2.adaptiveThreshold(scaled_frame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 121, 6) 

        license_no, license_no_conf = read_license_plate(frame)
        
        if license_no_conf is not None:
            license_no_conf = round(license_no_conf, 4)

            new_row = {'Car ID': car_id, 'License Plate Number': license_no, 'Frame Number': frame_no, 'Confidence Level': license_no_conf}
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index = True)
    
        processed_frames.add(filename)
    
    time.sleep(1)


# Original DF
excel_filename = "../Result/Detected License Plates.xlsx"
df.to_excel(excel_filename, index = False)
print(f"Original DataFrame saved to {excel_filename}")

# Sorted (Final) DF
df_sorted = df.sort_values(by = 'Confidence Level', ascending = False)
df_filtered = df_sorted.groupby('Car ID').first().reset_index()
excel_filename_2 = "../Result/Detected License Plates Sorted.xlsx"
df_filtered.to_excel(excel_filename_2, index = False)
print(f"Sorted DataFrame saved to {excel_filename_2}")

# Deleting Duplicate Plate Images
car_ids = set()
for filename in os.listdir(frames_folder):
    if filename.endswith(".jpg"):
        car_id = int(filename[filename.index('D')+1:filename.index('.')])
        if car_id not in car_ids:
            car_ids.add(car_id)
        else:
            frame_path = os.path.join(frames_folder, filename)
            os.remove(frame_path)

# Deleting Unused Car Images
car_images = set()
car_images_folder = "Detected Car Frames"
car_IDs_list = df['Car ID'].tolist()
for filename in os.listdir(car_images_folder):
    if filename.endswith(".jpg"):
        car_id = int(filename[filename.index('D')+1:filename.index('.')])
        if car_id not in car_IDs_list:
            car_image_path = os.path.join(car_images_folder, filename)
            os.remove(car_image_path)